# SPDE

In this tutorial, we show how to use the API SPDE.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import matplotlib.pyplot as plt

from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np


dat = gl.Db.create()
ndat = 1000
rangev = 0.2
sill = 1.
nugget = 0.1

np.random.seed(123)
dat["x"] = np.random.uniform(size=ndat)
dat["y"] = np.random.uniform(size=ndat)
dat.setLocators(["x","y"],gl.ELoc.X)

grid = gl.DbGrid.create([50,50],[0.02,0.02],[0,0])
gridExt = gl.DbGrid.create([75,75],[0.02,0.02],[-0.25,-0.25])
mesh = gl.MeshETurbo(gridExt)
model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=1,range=rangev,sill=sill)
model.addCovFromParam(gl.ECov.NUGGET,sill=nugget)

spde = gl.SPDE()
spde.init(model,grid,None,gl.ESPDECalcMode.SIMUNONCOND)


In [ ]:
spde.compute()
iuid = spde.query(dat)
iuid = spde.query(grid)

In [ ]:
ax = gp.grid(grid,title="Non Conditional Simulation")

In [ ]:
spdeRes = gl.SPDE()
spdeRes.init(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh)
spdeRes.compute()

In [ ]:
iuid = spdeRes.query(grid)
ax = gp.grid(grid, title="Estimation")

In [ ]:
Qtr = gl.csToTriplet(spdeRes.getPrecisionOp().getQ())
Pgl = gl.ProjMatrix(dat,mesh)
Apgl = Pgl.getAproj()
Atr = gl.csToTriplet(Apgl)
Q = sc.sparse.csc_matrix((np.array(Qtr.values), (np.array(Qtr.rows), np.array(Qtr.cols))))
Aproj = sc.sparse.csc_matrix((np.array(Atr.values), (np.array(Atr.rows), np.array(Atr.cols))),
                         shape=(Atr.nrows,Atr.ncols))

cholQ=cholesky(Q)
u = np.random.normal(size = Q.shape[0])
b = Q @ u
print(np.max(np.abs(cholQ.solve_A(b) - u)))


Qop = Q + 1/nugget * Aproj.T @ Aproj
cholQop =  cholesky(Qop)

In [ ]:
def solveMat(cholQop,x):
    return cholQop.solve_A(x)

def invSigma(sigma2,Aproj,cholQop,x):
    return 1./sigma2 * (x - 1./sigma2 * Aproj @ solveMat(cholQop, Aproj.T @ x))

def detQ(cholQ):
    return cholQ.logdet()

x = dat["spde.simu"]
ones = np.ones_like(x)
invSigmaOnes = invSigma(nugget,Aproj,cholQop,ones)
mu  = np.sum(x * invSigmaOnes)/np.sum( ones * invSigmaOnes) 
quad = np.sum((x-mu)*invSigma(nugget,Aproj,cholQop,x-mu))
logdet = len(x) * np.log(nugget) - detQ(cholQ) + detQ(cholQop)

print("logdet_chol",logdet)
print("quad_chol",quad)
print("like_chol",-0.5 * (quad + logdet))

In [ ]:
pcm = spdeRes.getPrecisionKriging()
a = pcm.computeLogDetOp(1,0)
b = detQ(cholQop)
print(a)
print(b)
print((b-a)/a)

In [ ]:
a = detQ(cholQ)
b = pcm.computeLogDetQ(10,0)
print(a)
print(b)
print((b-a)/a)

In [ ]:
a = spdeRes.computeLogLike(10,0)
b = -0.5 * (quad + logdet)
print(a)
print(b)
print((b-a)/b)

In [ ]:
pm = spdeRes.getPrecisionKriging()
x =[np.random.uniform(size=mesh.getNApices())]
y = gl.VectorVectorDouble()
y.push_back(gl.VectorDouble(mesh.getNApices()))
z = gl.VectorVectorDouble()
z.push_back(gl.VectorDouble(mesh.getNApices()))
pm.evalDirect(x,y)
pm.evalInverse(y,z)
#np.max(x[0]-z[0])

In [ ]:
ax = gp.plot(dat, size=2, aspect=1, figsize=[6,6])

In [ ]:
dat

In [ ]:
spdeRes = gl.SPDE()
spdeRes.init(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh)
spdeRes.compute()
spdeRes.query(grid)
spdeRes.computeQuad()

In [ ]:
gp.plot(grid, aspect=1, figsize=[6,6])

In [ ]:
from scipy.optimize import minimize
mesh = gl.MeshETurbo(gridExt)
class storage :
    def __init__(self):
        pass
            
    def f(self,x) :
        print(x**2)
        self.model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=1,
                                              range=x[0]**2,
                                              sill=x[1]**2)
        self.model.addCovFromParam(gl.ECov.NUGGET,sill=x[2]**2)
        self.spde = gl.SPDE()
        self.spde.init(self.model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh)
        self.spde.compute()
        res = -self.spde.computeLogLike(10,12134)
        print(res)
        return res
    
    def fchol(self,x):
        self.model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=1,
                                              range=x[0]**2,
                                              sill=x[1]**2)
        self.model.addCovFromParam(gl.ECov.NUGGET,sill=x[2]**2)
        self.spde = gl.SPDE()
        self.spde.init(self.model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh)
        self.spde.compute()
        Qtr = gl.csToTriplet(self.spde.getPrecisionOp().getQ())
        Pgl = gl.ProjMatrix(dat,mesh)
        Apgl = Pgl.getAproj()
        Atr = gl.csToTriplet(Apgl)
        Q = sc.sparse.csc_matrix((np.array(Qtr.values), (np.array(Qtr.rows), np.array(Qtr.cols))))
        Aproj = sc.sparse.csc_matrix((np.array(Atr.values), (np.array(Atr.rows), np.array(Atr.cols))),
                         shape=(Atr.nrows,Atr.ncols))
        cholQ=cholesky(Q)
        Qop = Q + 1/nugget * Aproj.T @ Aproj
        cholQop =  cholesky(Qop)
        x = dat["spde.simu"]
        ones = np.ones_like(x)
        invSigmaOnes = invSigma(nugget,Aproj,cholQop,ones)
        mu  = np.sum(x * invSigmaOnes)/np.sum( ones * invSigmaOnes) 
        quad = np.sum((x-mu)*invSigma(nugget,Aproj,cholQop,x-mu))
        logdet = len(x) * np.log(nugget) - detQ(cholQ) + detQ(cholQop)
        ones = np.ones_like(x)
        invSigmaOnes = invSigma(nugget,Aproj,cholQop,ones)
        mu  = np.sum(x * invSigmaOnes)/np.sum( ones * invSigmaOnes) 
        quad = np.sum((x-mu)*invSigma(nugget,Aproj,cholQop,x-mu))
        logdet = len(x) * np.log(nugget) - detQ(cholQ) + detQ(cholQop)
        return  0.5 * (logdet + quad)

In [ ]:
A = storage()
trueParams = np.array([np.sqrt(rangev),np.sqrt(sill),np.sqrt(nugget)])
print(trueParams**2)
A.f(trueParams)

In [ ]:
def cb(xk):
    print("-----------------------------------------------------")
    print(xk**2)
    
    
#paramsInit = np.sqrt([0.5,0.5,0.5])
#res = minimize(A.f, paramsInit,callback=cb,method="BFGS")


In [ ]:
trueParams

u = np.linspace(np.sqrt(0.5),np.sqrt(1.5),20)
res = [A.f(np.array([trueParams[0],i,trueParams[2]])) for i in u]
res2 = [A.fchol(np.array([trueParams[0],i,trueParams[2]])) for i in u]


In [ ]:
ax=plt.plot(u,-np.array(res2))
ax=plt.plot(u,-np.array(res))

#plt.vlines(1,-400,500)
print(u[np.argmin(res2)]**2)
print(u[np.argmin(res)]**2)

In [ ]:
# Vraisemblance 2D dans les 2 cas
